# Topic Modelling Notebook 

In [2]:
# Utilities
from joblib import Parallel, delayed
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
from multiprocessing import Pool, cpu_count
import warnings 
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
tqdm.pandas(desc="progress bar")
import gc
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

# Import time packages 
import time
import datetime

# Visualization Tools
import matplotlib.pyplot as plt

# Packages needed for text pre-processing:
import nltk 
import spacy 
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokens import Doc
from spacy.language import Language
import re
import contractions
import emoji
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Deep Learning Models 
import torch
import tensorflow as tf

# Sentiment Packages
from textblob import TextBlob
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import torch
from scipy.special import softmax


INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
from utils import search_folder
current_dir = os.getcwd()
thesis_folder_path = current_dir.replace("thesis_code", "")
twitter_data_path = search_folder(thesis_folder_path,"twitter_data")
eikon_data_path = search_folder(thesis_folder_path,"eikon_news")
stock_data_path = search_folder(thesis_folder_path,"stock_prices")
modelling_data_path = search_folder(thesis_folder_path,"modelling_data")
google_trending_path = search_folder(thesis_folder_path,"google_search")

In [4]:
# Retrieving sentiment on raw text 
twitter_senti_raw_df = pd.read_csv(twitter_data_path +"/twitter_senti_raw_df.csv")
twitter_senti_raw_df["datetime"] = pd.to_datetime(twitter_senti_raw_df["datetime"])
twitter_senti_raw_df =twitter_senti_raw_df.sort_values(by = "datetime")

In [5]:
# Retrieving sentiment on cleaned text
twitter_senti_df = pd.read_csv(twitter_data_path +"/twitter_senti_df.csv")
twitter_senti_df["datetime"] = pd.to_datetime(twitter_senti_df["datetime"])
twitter_senti_df =twitter_senti_df.sort_values(by = "datetime")
twitter_senti_df = twitter_senti_df.drop_duplicates()

In [6]:
# Retrieving Sentiment on Eikon Raw News 
eikon_senti_df = pd.read_csv(eikon_data_path +"/raw_eikon_senti_df.csv")
eikon_senti_df["datetime"] = pd.to_datetime(eikon_senti_df["datetime"])
eikon_senti_df =eikon_senti_df.sort_values(by = "datetime")

In [7]:
print(cleaned_tweets_df.shape)
print(twitter_senti_raw_df.shape)
print(twitter_senti_df.shape)
print(eikon_senti_df.shape)

NameError: name 'cleaned_tweets_df' is not defined

In [ ]:
cleaned_tweets_df.datetime.dt.day_name().value_counts()

datetime
Tuesday      387865
Wednesday    385878
Thursday     376579
Friday       346848
Monday       343058
Saturday     166083
Sunday       151677
Name: count, dtype: int64

In [ ]:
eikon_senti_df.datetime.dt.day_name().value_counts()

datetime
Wednesday    9026
Thursday     8827
Tuesday      8635
Friday       7746
Monday       7096
Saturday     1983
Sunday       1110
Name: count, dtype: int64

In [ ]:
# Tweets on Saturday and Sunday are moved to Monday in order to not losse any relevance
from utils import fix_dates
twitter_senti_raw_df["datetime"] = twitter_senti_raw_df["datetime"].progress_apply(lambda x: fix_dates(x))
twitter_senti_df["datetime"] = twitter_senti_df["datetime"].progress_apply(lambda x: fix_dates(x))
eikon_senti_df["datetime"] = eikon_senti_df["datetime"].progress_apply(lambda x: fix_dates(x))

progress bar: 100%|██████████| 44423/44423 [00:00<00:00, 233269.23it/s]
